# <font color='crimson'>1. 모듈 가져오기</font>

### <font color='darkcyan'>Jetson 관련 Modules</font>

In [1]:
import jetson.inference
import jetson.utils
from segnet_utils import *
import argparse
from jetbot import Camera

### <font color='darkcyan'>이미지 처리 및 출력 Modules</font>

In [2]:
import cv2
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg

---
<br>

# <font color='crimson'>2. 이미지 가져오기 & 정보 출력</font>

In [4]:
img = jetson.utils.loadImage('/home/jetbot/works/jetson-inference/build/aarch64/bin/images/city_0.jpg')
print(img.shape, type(img))  # (565, 563, 3)
print(img.width, img.height, img.channels)

(512, 1024, 3) <class 'jetson.utils.cudaImage'>
1024 512 3


---
<br>

# <font color='crimson'>3. Inference</font>

### <font color='darkcyan'>사전 학습된 모델 가져오기</font>
* 처음 모델을 가져올 때 많은 시간이 소요됨

In [3]:
net = jetson.inference.segNet('fcn-resnet18-cityscapes')

### <font color='darkcyan'>예측</font>

In [33]:
args = argparse.Namespace()
args.stats='store_false'               # 통계 저장 여부
args.visualize = 'mask,overlay'        # 출력 결과
args.alpha = 150.0                     # alpha blending: 0 - 255

buffers = segmentationBuffers(net, args) # 픽셀 정보를 저장할 버퍼

buffers.Alloc(img.shape, img.format)

net.SetOverlayAlpha(args.alpha)
net.Process(img)

net.Overlay(buffers.overlay, filter_mode='linear')  # linear | point
net.Mask(buffers.mask, filter_mode='linear')

jetson.utils.cudaOverlay(buffers.overlay, buffers.composite, 0, 0)
jetson.utils.cudaOverlay(buffers.mask, buffers.composite, buffers.overlay.width, 0)

segImg = buffers.output


---
<br>

# <font color='crimson'>4. 이미지 출력</font>

### <font color='darkcyan'>이미지 변환</font>

In [34]:
image = jetson.utils.cudaToNumpy(segImg )  #cudaImage 형식을 numpy로 변환
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #OpenCV에서 사용하는 BGR 형태의 numpy로 변환
image = bgr8_to_jpeg(image)  #numpy를 이미지로 변환

### <font color='darkcyan'>이미지 출력 위젯 생성</font>

In [35]:
widget = widgets.Image(format='jpeg', width=300, height=300)

### <font color='darkcyan'>이미지 출력</font>

In [36]:
widget.value = image
display(widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

---
<br>

# <font color='crimson'>5. 카메라 활용한 분류</font>

In [3]:
net = jetson.inference.segNet('fcn-resnet18-sun')

### <font color='darkcyan'>카메라 설정</font>

In [4]:
camera = Camera.instance(width=300, height=300, fps=1)

### <font color='darkcyan'>이미지 출력 위젯 설정</font>

In [5]:
widget = widgets.Image(format='jpeg', width=300, height=300)
display(widget)

Image(value=b'', format='jpeg', height='300', width='300')

In [6]:
while 1:
    try:
        frame = camera.value
        img_cuda = jetson.utils.cudaFromNumpy(frame)
        
        args = argparse.Namespace()
        args.stats='store_false'               # 통계 저장 여부
        args.visualize = 'mask,overlay'        # 출력 결과
        args.alpha = 150.0                     # alpha blending: 0 - 255

        buffers = segmentationBuffers(net, args) # 픽셀 정보를 저장할 버퍼

        buffers.Alloc(img_cuda.shape, img_cuda.format)

        net.SetOverlayAlpha(args.alpha)
        net.Process(img_cuda)

        net.Overlay(buffers.overlay, filter_mode='linear')  # linear | point
        net.Mask(buffers.mask, filter_mode='linear')

        jetson.utils.cudaOverlay(buffers.overlay, buffers.composite, 0, 0)
        jetson.utils.cudaOverlay(buffers.mask, buffers.composite, buffers.overlay.width, 0)

        segImg = buffers.output

        image = jetson.utils.cudaToNumpy(segImg)
        widget.value = bgr8_to_jpeg(image)
    except KeyboardInterrupt:
        break

camera.stop()
del camera